In [1]:
# !pip install mlflow

In [15]:
# !npm install -g localtunnel

/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/lt.js
+ localtunnel@2.0.2
added 22 packages from 22 contributors in 2.124s


In [1]:
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from pathlib import Path
import numpy as np
import mlflow

2022-12-09 06:35:23.050854: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-09 06:35:23.253804: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-09 06:35:23.253835: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-09 06:35:23.279128: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-09 06:35:24.112811: W tensorflow/stream_executor/platform/de

In [2]:
'''When creating new experiment'''
# experiment_id = mlflow.create_experiment(
#     "MNIST MLP classifier", 
#     artifact_location=Path.cwd().joinpath("./mlruns").as_uri(),
#     tags={"version":"v1", "priority":"P1"}
# )

'When creating new experiment'

In [4]:
mlflow.set_tracking_uri("sqlite:///mlruns/mlruns.db")

In [5]:
experiment = mlflow.set_experiment("MNIST MLP classifier")

In [7]:
(train_images,train_labels),(test_images,test_labels) = mnist.load_data()
print(len(train_images))

60000


In [8]:
optim = "rmsprop"
epochs = 5
batch_size = 128

In [9]:
model = keras.Sequential([layers.Dense(128,activation='relu'),
layers.Dense(10,activation='softmax')])
model.compile(optimizer=optim,loss='sparse_categorical_crossentropy',metrics=['accuracy'])

2022-12-09 06:36:17.131900: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-12-09 06:36:17.131960: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-09 06:36:17.131989: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (f0a27f9ea0cc): /proc/driver/nvidia/version does not exist
2022-12-09 06:36:17.132316: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [10]:
train_images = train_images.reshape((60000,28*28))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000,28*28))
test_images = test_images.astype("float32") / 255

In [11]:
with mlflow.start_run(experiment_id=experiment.experiment_id, description="MLP model for classifying MNIST hand written digits", run_name=f"{optim}-e{epochs}-{batch_size}") as run:
  mlflow.keras.autolog(registered_model_name="MLP MNIST classifier")
  history = model.fit(train_images,train_labels,epochs = epochs, batch_size = batch_size)

/usr/local/lib/python3.8/dist-packages/mlflow/utils/autologging_utils/__init__.py:416: FutureWarning: Autologging support for keras >= 2.6.0 has been deprecated and will be removed in a future MLflow release. Use `mlflow.tensorflow.autolog()` instead.
  return _autolog(*args, **kwargs)
2022/12/09 06:36:21 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during autologging: This model has not yet been built. Build the model first by calling `build()` or by calling the model on a batch of data.


Epoch 1/5
469/469 [==============================] - 2s 3ms/step - loss: 0.3342 - accuracy: 0.9083
Epoch 2/5
469/469 [==============================] - 1s 3ms/step - loss: 0.1546 - accuracy: 0.9561
Epoch 3/5
469/469 [==============================] - 1s 3ms/step - loss: 0.1093 - accuracy: 0.9690
Epoch 4/5
469/469 [==============================] - 1s 3ms/step - loss: 0.0844 - accuracy: 0.9755
Epoch 5/5
469/469 [==============================] - 2s 3ms/step - loss: 0.0691 - accuracy: 0.9797
INFO:tensorflow:Assets written to: /tmp/tmplveuabgl/model/data/model/assets


Registered model 'MLP MNIST classifier' already exists. Creating a new version of this model...
2022/12/09 06:36:34 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: MLP MNIST classifier, version 7
Created version '7' of model 'MLP MNIST classifier'.


In [39]:
# get_ipython().system_raw("mlflow server --host 0.0.0.0 -p 8000 --backend-store-uri sqlite:///mlruns/mlruns.db --default-artifact-root ./mlruns &")

In [40]:
# get_ipython().system_raw('lt --port 8000 >> url.txt 2>&1 &')

In [41]:
# !rm url.txt

In [42]:
# !cat url.txt

In [12]:
test_img = np.expand_dims(test_images[0][:], axis=0)
test_labl = test_labels[0]
pred_labl = model.predict(test_img)

1/1 [==============================] - 0s 97ms/step


In [13]:
# print(f"Predicted error {test_labl[0] - pred_labl[0].argmax()}") 

In [14]:
# test_loss, test_accuracy = model.evaluate(test_img,test_labl)
# print('Test Loss: ', test_loss)
# print('Test Accuracy: ',test_accuracy*100)

In [15]:
test_loss, test_accuracy = model.evaluate(test_images,test_labels)
print('Test Loss: ', test_loss)
print('Test Accuracy: ',test_accuracy*100)

313/313 [==============================] - 1s 2ms/step - loss: 0.0832 - accuracy: 0.9764
Test Loss:  0.08323520421981812
Test Accuracy:  97.64000177383423


## Loading model for predictions

In [16]:
run.info.run_id

'5fce44ef4b33403e8546d9973dc8b1c7'

In [17]:
keras_model = mlflow.keras.load_model(f"runs:/{run.info.run_id}" + "/model")

In [18]:
predictions = keras_model.predict(test_img)

1/1 [==============================] - 0s 59ms/step


In [19]:
predictions[0].argmax()

7

In [20]:
test_images[0].shape

(784,)

## Infering with served model

In [35]:
get_ipython().system_raw(f"mlflow models serve --model-uri mlruns/{experiment.experiment_id}/{run.info.run_id}/artifacts/model --no-conda &")

/usr/local/lib/python3.8/dist-packages/click/core.py:2309: FutureWarning: `--no-conda` is deprecated and will be removed in a future MLflow release. Use `--env-manager=local` instead.
  value = self.callback(ctx, self, value)
2022/12/09 07:11:14 INFO mlflow.models.cli: Selected backend for flavor 'python_function'
2022/12/09 07:11:14 INFO mlflow.pyfunc.backend: === Running command 'exec gunicorn --timeout=60 -b 127.0.0.1:5000 -w 1 ${GUNICORN_CMD_ARGS} -- mlflow.pyfunc.scoring_server.wsgi:app'
[2022-12-09 07:11:14 +0000] [610] [INFO] Starting gunicorn 20.1.0
[2022-12-09 07:11:14 +0000] [610] [INFO] Listening at: http://127.0.0.1:5000 (610)
[2022-12-09 07:11:14 +0000] [610] [INFO] Using worker: sync
[2022-12-09 07:11:14 +0000] [612] [INFO] Booting worker with pid: 612
2022-12-09 07:11:16.021907: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-crit

In [36]:
import requests
import json
import numpy as np

In [37]:
json_data = {'data': np.expand_dims(test_images[0], axis=0).tolist()}

In [38]:
headers = {'Content-type': 'application/json'}
res = requests.post('http://127.0.0.1:5000/invocations', headers=headers, data=json.dumps(json_data))
print(res.text)

1/1 [==============================] - 0s 77ms/step
[{"0": 2.0703350855910685e-07, "1": 1.0532378302841039e-09, "2": 3.158929030178115e-05, "3": 0.00036576087586581707, "4": 2.667395859745625e-09, "5": 3.588436072732293e-07, "6": 2.431265041721864e-12, "7": 0.9995895028114319, "8": 9.836113349592779e-06, "9": 2.7452881568024168e-06}]
